In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import sentencepiece as spm
import gensim
from gensim.models import Word2Vec
import random
from tqdm import tqdm
import json
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from two_tower_classes import TwoTowerNN

/Users/beesamprajveenkumar/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vocab_size = 1000
embedding_dim = 100
query_hidden_size = 64
doc_hidden_size = 64
query_num_layers = 2
doc_num_layers = 2
output_size = 1
fine_tune=False

eval_model = TwoTowerNN( 
    sp_model_path='m.model', word2vec_model_path='word2vec.model',
    embedding_dim=embedding_dim,
    query_hidden_size=query_hidden_size, doc_hidden_size=doc_hidden_size,
    query_num_layers=query_num_layers, doc_num_layers=doc_num_layers,
    output_size=output_size, vocab_size=vocab_size, fine_tune=fine_tune,
)
eval_model.load_state_dict(torch.load('two_tower_model.pth'))

<All keys matched successfully>

In [4]:
# Load document embeddings from the file
with open("document_embeddings.json", "r") as f:
    document_embeddings_lists = json.load(f)

# Convert lists to tensors
document_embeds = {i: torch.tensor(embedding) for i, embedding in enumerate(document_embeddings_lists)}

In [5]:
# Load the dictionary from the JSON file
with open("id_doc_dict.json", "r") as f:
    id_doc_dict = json.load(f)

In [6]:
# predict
query='What is rba?'
candidate_docs_ids=id_doc_dict
k=5
result = eval_model.predict(
    query, candidate_docs_ids, id_doc_dict=id_doc_dict,
    doc_emb_dict=document_embeds, k=k,
)



def predict_passages(query):
    candidate_docs_ids=id_doc_dict
    k=5
    result = eval_model.predict(query, candidate_docs_ids, id_doc_dict=id_doc_dict, doc_emb_dict=document_embeds, k=k)
    return result



/Users/beesamprajveenkumar/Documents/Projects/MLX/two_tower/app/model/two_tower_classes.py:140: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  query_input = torch.tensor(query_input)  # Convert to tensor if required


In [7]:
result

[("Bachelor of science in biology degree holders working as research laboratory technicians earned $28,000 to $38,000 per year, according to a survey of 1,303 biology majors by PayScale.com that was updated in November 2010. Biology majors working as high school teachers made salaries between $36,801 and $48,519 annually. Mid-Career Income. Molecular biology majors with 15 years of experience earned a median salary of $82,900 per year, according to PayScale.com's College Salary Report, while those with a general biology degree earned $72,800 a year, just ahead of microbiology majors, whose median mid-career salary was $72,600.",
  0.14949828386306763),
 ("Starting Salaries. The starting median pay for biology majors was $40,600 per year for those who specialized in microbiology, according to PayScale.com's 2010-2011 College Salary Report, while for those with a molecular biology focus it was $40,200 annually. General biology majors had a median starting salary of $38,400. Mid-Career In

In [8]:
answer1 = predict_passages('welcome to england')

In [9]:
answer1

[("Bachelor of science in biology degree holders working as research laboratory technicians earned $28,000 to $38,000 per year, according to a survey of 1,303 biology majors by PayScale.com that was updated in November 2010. Biology majors working as high school teachers made salaries between $36,801 and $48,519 annually. Mid-Career Income. Molecular biology majors with 15 years of experience earned a median salary of $82,900 per year, according to PayScale.com's College Salary Report, while those with a general biology degree earned $72,800 a year, just ahead of microbiology majors, whose median mid-career salary was $72,600.",
  0.1706259399652481),
 ("Starting Salaries. The starting median pay for biology majors was $40,600 per year for those who specialized in microbiology, according to PayScale.com's 2010-2011 College Salary Report, while for those with a molecular biology focus it was $40,200 annually. General biology majors had a median starting salary of $38,400. Mid-Career Inc

In [11]:
answer1.type

AttributeError: 'list' object has no attribute 'type'